In [1]:
from google.colab import drive
from os import listdir
from os.path import isfile, join
from scipy.io import loadmat
from random import randint
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report

import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
import pandas as pd
import numpy as np
import random

import datetime
import os

In [2]:
# Establecemos conexión con Google Drive para acceder a la carpeta de datasets
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Guardamos la dirección de los datasets (experimentos) en una variable para utilizala posteriormente
dataset_datos_dir = 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS'

In [4]:
# Obtenemos una lista con todos los nombres de los archivos relacionados a los experimentos realizados
dataset_datos_files = [dataset_datos_dir+'/'+filename for filename in listdir(dataset_datos_dir) if isfile(join(dataset_datos_dir, filename))]

In [5]:
dataset_datos_files

['drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS/1_1_05A.mat',
 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS/1_5_05A.mat',
 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS/1_2_3A.mat',
 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS/1_5_2A.mat',
 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS/1_3_3A.mat',
 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS/1_4_2A.mat',
 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS/1_5_1A.mat',
 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS/1_1_3A.mat',
 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS/1_3_05A.mat',
 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS/1_2_05A.mat',
 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS/1_10_05A.mat',
 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS/1_10_3A.mat',
 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS/1_3_1A.mat',
 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS/1_4_3A.mat',
 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS/1_1_1A.mat',
 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATO

In [6]:
# Creamos listas en donde se almacenarán los archivos relacionados a experimentos de fisura y experimentos de perno flojo respectivamente
datos_fisura = []
datos_pernoflojo = []
datos_replica = []
datos_normal = []

In [7]:
# Iteramos los nombres de todos los archivos en la lista general previamente creada
for filename in listdir(dataset_datos_dir):
  # Obtenemos el nombre del archivo tal cual, sin direcciones previas
  if isfile(join(dataset_datos_dir, filename)):
    nombre_archivo = filename.split('.')[0]
    # El estado de fisura se asocia al número 3 en los nombres de los archivos, si cumple la condición el archivo, guardamos en la lista datos_fisura
    if (nombre_archivo.split('_')[0] == '3'):
      datos_fisura.append(dataset_datos_dir + '/' + filename)
    # El estado de fisura se asocia al número 4 en los nombres de los archivos, si cumple la condición el archivo, guardamos en la lista datos_pernoflojo
    elif (nombre_archivo.split('_')[0] == '4'):
      datos_pernoflojo.append(dataset_datos_dir + '/' + filename)
    # El estado de fisura se asocia al número 4 en los nombres de los archivos, si cumple la condición el archivo, guardamos en la lista datos_pernoflojo
    elif (nombre_archivo.split('_')[0] == '1'):
      datos_normal.append(dataset_datos_dir + '/' + filename)
    # El estado de fisura se asocia al número 4 en los nombres de los archivos, si cumple la condición el archivo, guardamos en la lista datos_pernoflojo
    elif (nombre_archivo.split('_')[0] == '2'):
      datos_replica.append(dataset_datos_dir + '/' + filename)

In [8]:
len(datos_normal)

40

In [9]:
len(datos_replica)

20

In [10]:
len(datos_fisura)

20

In [11]:
len(datos_pernoflojo)

20

## Datos Fisura

In [12]:
# Se crean 2 listas, una asociada al número de experimento y otra asociada al nivel de white noise presente durante el experimento 
numero_experimentos_fisura = []
amplitud_experimentos_fisura = []

In [13]:
# Iteramos los nombres de los archivos dentro del conjunto que contiene los datos de fisura
for nombre_archivo in datos_fisura:
  # Obtenemos el nombre del experimento
  nombre_experimento = nombre_archivo.split('/')[4]
  # Obtenemos el número del experimento
  numero_experimento = nombre_experimento.split('_')[1]
  # Obtenemos el nivel de white noise del experimento
  amplitud_experimento = nombre_experimento.split('.')[0].split('_')[2].replace('A', '')
  # Agregamos el número del experimento a la lista numero_experimentos_fisura creada anteriormente
  numero_experimentos_fisura.append(int(numero_experimento))
  # Agregamos el nivel de white noise del experimento a la lista amplitud_experimentos_fisura creada anteriormente
  if amplitud_experimento == '05':
    amplitud_experimento = 0.5
    amplitud_experimentos_fisura.append(amplitud_experimento)
  else:
    amplitud_experimentos_fisura.append(int(amplitud_experimento))

In [14]:
# Creamos un DataFrame vacío
df_fisura = pd.DataFrame()

In [15]:
# Iteramos por rango la lista datos_fisura
for indice in range(len(datos_fisura)):
  # Obtenemos la dirección dentro de la iteración correspondiente
  direccion = datos_fisura[indice]
  # Transformamos el .mat a un formato en el que se pueda transformar a DataFrame
  mat = loadmat(direccion)
  df = pd.DataFrame(mat['data'])
  # Para todos los valores del experimento, creamos columnas en donde sus valores se asocian con el nivel de wn y el # de experimento
  df['#_exp'] = numero_experimentos_fisura[indice]
  df['amplitud'] = amplitud_experimentos_fisura[indice]
  # Colocamos al final del DataFrame df_fisura creado el nuevo DataFrame 
  df_fisura = pd.concat([df_fisura, df], axis = 0)

In [16]:
# Revisamos el conjunto de datos obtenido para fisura
df_fisura.head(8)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,#_exp,amplitud
0,0.000132,0.000185,0.000122,0.000105,0.000141,0.000075,0.000007,0.000231,0.000299,0.000113,0.000228,0.000225,0.000168,0.000018,0.000344,0.000275,-0.000017,0.000046,0.000200,0.000169,0.000097,0.000196,-0.000053,0.000018,1,1.0
1,0.000092,0.000223,0.000166,0.000127,0.000191,0.000137,0.000075,0.000192,0.000240,0.000095,0.000279,0.000231,0.000145,0.000037,0.000339,0.000092,0.000085,0.000074,0.000128,0.000163,0.000074,0.000085,0.000063,-0.000027,1,1.0
2,0.000091,0.000280,0.000188,0.000130,0.000172,0.000102,0.000161,0.000158,0.000211,0.000123,0.000269,0.000206,0.000198,0.000041,0.000312,0.000179,0.000011,0.000079,0.000117,0.000146,0.000085,0.000089,-0.000014,0.000020,1,1.0
3,0.000179,0.000199,0.000249,0.000102,0.000268,0.000140,0.000126,0.000231,0.000239,0.000102,0.000284,0.000175,0.000164,0.000082,0.000311,0.000235,0.000031,0.000079,0.000257,0.000110,0.000050,0.000120,0.000080,0.000057,1,1.0
4,0.000282,0.000277,0.000202,0.000211,0.000251,0.000159,0.000096,0.000205,0.000342,0.000113,0.000240,0.000272,0.000214,0.000057,0.000304,0.000302,0.000047,0.000150,0.000271,0.000168,0.000126,0.000224,0.000105,0.000058,1,1.0
5,0.000005,0.000196,0.000247,0.000109,0.000186,0.000115,0.000042,0.000129,0.000255,0.000108,0.000315,0.000214,0.000237,-0.000001,0.000334,0.000202,-0.000008,0.000081,0.000173,0.000165,0.000126,0.000107,-0.000012,0.000006,1,1.0
6,0.000176,0.000372,0.000223,0.000154,0.000258,0.000165,0.000170,0.000157,0.000228,0.000126,0.000282,0.000219,0.000198,0.000081,0.000333,0.000146,0.000065,0.000070,0.000140,0.000145,0.000099,0.000115,0.000068,-0.000031,1,1.0
7,0.000133,0.000171,0.000187,0.000182,0.000237,0.000159,0.000121,0.000280,0.000293,0.000096,0.000248,0.000202,0.000160,0.000015,0.000319,0.000335,-0.000004,0.000102,0.000201,0.000193,0.000180,0.000171,-0.000009,0.000095,1,1.0


In [17]:
# Si dividimos para 99097 el df_fisura deberíamos obtener 20 experimentos en total
len(df_fisura)/99097

20.0

### Procesamiento de Datos

In [18]:
# Establecemos 2 listas, una con los posibles experimentos y otra con los posibles niveles de white noise, lo cual nos ayudará a segmentar los datos
experimentos = [1, 2, 3, 4, 5]
wns = [0.5, 1, 2, 3]
# Creamos una lista en donde pondremos todas las imágenes creadas, segmentadas por experimento y white noise
arreglo_imagenes_por_experimento_fisura = []

# Iteramos por experimento y por nivel de white noise
for i in experimentos:
  for j in wns:
    # Creamos filtros que nos permiten identificar el experimento por su número asociado y por su nivel de white noise dentro del conjunto de datos
    filter1 = df_fisura["#_exp"] == i
    filter2 = df_fisura["amplitud"] == j
    # Filtramos y deshacemos las filas que no corresponden a la búsqueda
    dataset_experimento = df_fisura.where(filter1 & filter2).dropna()
    # Creamos arreglo en donde almacenaremos listas de imágenes asociadas a cada columna en un experimento individual
    arreglo_matrices_asociadas = []

    # En el conjunto filtrado, iteramos por cada columna (de la columna 0 a la 23)
    for n in range(24):
      # Establecemos un límite inferior y un límite superior
      indice_inicio = 0
      indice_fin = 256
      # Del conjunto filtrado, obtenemos un subconjunto con muestras cada 6 pasos
      columna_cada_6 = dataset_experimento[n][::6]
      # Creamos una lista en donde guardaremos todas las imágenes creadas dentro de la columna en la que se esta iterando
      matrices_columna = []
    
      # Recorremos en valores de 256 la columna del subconjunto creado para generar las imágenes 16x16 correspondientes 
      while indice_fin < len(columna_cada_6):
        # Guardamos el conjunto de 256 datos dentro de un arreglo
        vector = columna_cada_6[indice_inicio:indice_fin]
        # Redimensionamos el arreglo (de 1x256 a 16x16)
        matriz = vector.to_numpy().reshape((16, 16))
        # Agregamos la imagen dentro de la lista matrices_columna
        matrices_columna.append(matriz)
        # Establecemos el nuevo límite inferior y superior en donde se encontrarán los datos de mi nueva imagen 16x16
        indice_inicio = indice_fin
        indice_fin = indice_fin + 256
      
      # Agregamos las listas de imágenes 16x16 asociadas a una columna a arreglo_matrices_asociadas
      arreglo_matrices_asociadas.append(matrices_columna)
    
    # Agregamos arreglo_matrices_asociadas a arreglos_imagenes_por_experimento, en donde tenemos listas de listas asociadas a un experimento individual
    arreglo_imagenes_por_experimento_fisura.append(arreglo_matrices_asociadas)

In [19]:
len(arreglo_imagenes_por_experimento_fisura)

20

In [20]:
len(arreglo_imagenes_por_experimento_fisura[0])

24

In [21]:
len(arreglo_imagenes_por_experimento_fisura[0][0])

64

In [22]:
arreglo_imagenes_por_experimento_fisura[0][0][0].size

256

In [23]:
arreglo_imagenes_por_experimento_fisura[0][0][0].shape

(16, 16)

In [24]:
# Creamos un arreglos general para todas las imágenes de dimensionamiento (24, 16, 16)
arreglo_imagenes_fisura = []

# Iteramos cada experimento en la lista de experimentos creada previamente
for experimento in arreglo_imagenes_por_experimento_fisura:
  # Iteramos en un rango de 64 para usarlo posteriormente
  for idx in range(64):
    # Para cada iteración se crea el arreglo en donde se agregaran las capas correctas de la imagen de dimensionamniento (24, 16, 16)
    imagen = []
    # Iteramos para cada fila en el experimento (son 24 filas)
    for fila in experimento:
      # Asociamos por índice los valores de cada lista de imágenes para establecer la relación correcta y agregamos al arreglo creado
      capa_imagen = fila[idx]
      imagen.append(capa_imagen)

    # Transformamos la lista de 24 capas a un arreglo numérico
    arreglo_imagenes_fisura.append(np.array(imagen))

In [25]:
# Revisamos el número de imágenes creadas
len(arreglo_imagenes_fisura)

1280

In [26]:
# Revisamos el dimensionamiento y tipo de un elemento aleatorio en la lista de imágenes
indice_imagen_fisura = randint(0, len(arreglo_imagenes_fisura) - 1)
arreglo_imagenes_fisura[indice_imagen_fisura].shape, type(arreglo_imagenes_fisura[0]), indice_imagen_fisura

((24, 16, 16), numpy.ndarray, 472)

## Datos Perno Flojo

In [27]:
# Se crean 2 listas, una asociada al número de experimento y otra asociada al nivel de white noise presente durante el experimento 
numero_experimentos_pernoflojo = []
amplitud_experimentos_pernoflojo = []

In [28]:
# Iteramos los nombres de los archivos dentro del conjunto que contiene los datos de fisura
for nombre_archivo in datos_pernoflojo:
  # Obtenemos el nombre del experimento
  nombre_experimento = nombre_archivo.split('/')[4]
  # Obtenemos el número del experimento
  numero_experimento = nombre_experimento.split('_')[1]
  # Obtenemos el nivel de white noise del experimento
  amplitud_experimento = nombre_experimento.split('.')[0].split('_')[2].replace('A', '')
  # Agregamos el número del experimento a la lista numero_experimentos_pernoflojo creada anteriormente
  numero_experimentos_pernoflojo.append(int(numero_experimento))
  # Agregamos el nivel de white noise del experimento a la lista amplitud_experimentos_pernoflojo creada anteriormente
  if amplitud_experimento == '05':
    amplitud_experimento = 0.5
    amplitud_experimentos_pernoflojo.append(amplitud_experimento)
  else:
    amplitud_experimentos_pernoflojo.append(int(amplitud_experimento))

In [29]:
# Creamos un DataFrame vacío
df_pernoflojo = pd.DataFrame()

In [30]:
# Iteramos por rango la lista datos_fisura
for indice in range(len(datos_pernoflojo)):
  # Obtenemos la dirección dentro de la iteración correspondiente
  direccion = datos_pernoflojo[indice]
  # Transformamos el .mat a un formato en el que se pueda transformar a DataFrame
  mat = loadmat(direccion)
  df = pd.DataFrame(mat['data'])
  # Para todos los valores del experimento, creamos columnas en donde sus valores se asocian con el nivel de wn y el # de experimento
  df['#_exp'] = numero_experimentos_pernoflojo[indice]
  df['amplitud'] = amplitud_experimentos_pernoflojo[indice]
  # Colocamos al final del DataFrame df_fisura creado el nuevo DataFrame 
  df_pernoflojo = pd.concat([df_pernoflojo, df], axis = 0)

In [31]:
# Revisamos el conjunto de datos obtenido para pernoflojo
df_pernoflojo.head(8)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,#_exp,amplitud
0,0.000168,0.000204,0.000213,0.000135,0.000201,0.000099,0.000108,0.000189,0.000212,0.000093,0.000257,0.000196,0.000173,0.000029,0.000316,0.000202,0.000010,0.000085,0.000151,0.000136,0.000116,0.000099,0.000054,0.000111,1,0.5
1,0.000122,0.000277,0.000213,0.000180,0.000230,0.000172,0.000068,0.000213,0.000214,0.000095,0.000272,0.000172,0.000173,0.000050,0.000292,0.000178,0.000021,0.000022,0.000203,0.000139,0.000076,0.000164,0.000089,0.000076,1,0.5
2,0.000099,0.000214,0.000128,0.000153,0.000217,0.000100,0.000064,0.000174,0.000225,0.000116,0.000245,0.000202,0.000140,0.000012,0.000309,0.000256,0.000023,0.000089,0.000225,0.000162,0.000098,0.000154,0.000091,0.000092,1,0.5
3,0.000150,0.000258,0.000209,0.000163,0.000213,0.000120,0.000066,0.000157,0.000220,0.000096,0.000229,0.000203,0.000157,0.000057,0.000292,0.000260,0.000062,0.000077,0.000212,0.000178,0.000123,0.000150,0.000056,0.000130,1,0.5
4,0.000194,0.000244,0.000194,0.000201,0.000202,0.000127,0.000133,0.000197,0.000212,0.000110,0.000266,0.000199,0.000181,0.000040,0.000336,0.000209,0.000092,0.000097,0.000183,0.000170,0.000114,0.000127,0.000064,0.000087,1,0.5
5,0.000156,0.000287,0.000188,0.000119,0.000234,0.000127,0.000102,0.000198,0.000216,0.000097,0.000280,0.000222,0.000140,0.000058,0.000296,0.000212,0.000061,0.000097,0.000231,0.000156,0.000090,0.000155,0.000076,0.000091,1,0.5
6,0.000150,0.000230,0.000198,0.000239,0.000247,0.000150,0.000052,0.000178,0.000255,0.000115,0.000235,0.000189,0.000216,0.000072,0.000299,0.000233,0.000060,0.000124,0.000225,0.000157,0.000133,0.000153,0.000060,0.000087,1,0.5
7,0.000130,0.000230,0.000207,0.000116,0.000212,0.000108,0.000096,0.000178,0.000200,0.000130,0.000260,0.000203,0.000216,0.000057,0.000329,0.000261,0.000031,0.000045,0.000236,0.000178,0.000081,0.000160,0.000079,0.000115,1,0.5


In [32]:
# Si dividimos para 99097 el df_pernoflojo deberíamos obtener 20 experimentos en total
len(df_pernoflojo)/99097

20.0

### Procesamiento de datos

In [33]:
# Establecemos 2 listas, una con los posibles experimentos y otra con los posibles niveles de white noise, lo cual nos ayudará a segmentar los datos
experimentos = [1, 2, 3, 4, 5]
wns = [0.5, 1, 2, 3]
# Creamos una lista en donde pondremos todas las imágenes creadas, segmentadas por experimento y white noise
arreglo_imagenes_por_experimento_pernoflojo = []

# Iteramos por experimento y por nivel de white noise
for i in experimentos:
  for j in wns:
    # Creamos filtros que nos permiten identificar el experimento por su número asociado y por su nivel de white noise dentro del conjunto de datos
    filter1 = df_pernoflojo["#_exp"] == i
    filter2 = df_pernoflojo["amplitud"] == j
    # Filtramos y deshacemos las filas que no corresponden a la búsqueda
    dataset_experimento = df_pernoflojo.where(filter1 & filter2).dropna()
    # Creamos arreglo en donde almacenaremos listas de imágenes asociadas a cada columna en un experimento individual
    arreglo_matrices_asociadas = []

    # En el conjunto filtrado, iteramos por cada columna (de la columna 0 a la 23)
    for n in range(24):
      # Establecemos un límite inferior y un límite superior
      indice_inicio = 0
      indice_fin = 256
      # Del conjunto filtrado, obtenemos un subconjunto con muestras cada 6 pasos
      columna_cada_6 = dataset_experimento[n][::6]
      # Creamos una lista en donde guardaremos todas las imágenes creadas dentro de la columna en la que se esta iterando
      matrices_columna = []
    
      # Recorremos en valores de 256 la columna del subconjunto creado para generar las imágenes 16x16 correspondientes 
      while indice_fin < len(columna_cada_6):
        # Guardamos el conjunto de 256 datos dentro de un arreglo
        vector = columna_cada_6[indice_inicio:indice_fin]
        # Redimensionamos el arreglo (de 1x256 a 16x16)
        matriz = vector.to_numpy().reshape((16, 16))
        # Agregamos la imagen dentro de la lista matrices_columna
        matrices_columna.append(matriz)
        # Establecemos el nuevo límite inferior y superior en donde se encontrarán los datos de mi nueva imagen 16x16
        indice_inicio = indice_fin
        indice_fin = indice_fin + 256
      
      # Agregamos las listas de imágenes 16x16 asociadas a una columna a arreglo_matrices_asociadas
      arreglo_matrices_asociadas.append(matrices_columna)
    
    # Agregamos arreglo_matrices_asociadas a arreglos_imagenes_por_experimento, en donde tenemos listas de listas asociadas a un experimento individual
    arreglo_imagenes_por_experimento_pernoflojo.append(arreglo_matrices_asociadas)

In [34]:
len(arreglo_imagenes_por_experimento_pernoflojo)

20

In [35]:
len(arreglo_imagenes_por_experimento_pernoflojo[0])

24

In [36]:
len(arreglo_imagenes_por_experimento_pernoflojo[0][0])

64

In [37]:
arreglo_imagenes_por_experimento_pernoflojo[0][0][0].size

256

In [38]:
arreglo_imagenes_por_experimento_pernoflojo[0][0][0].shape

(16, 16)

In [39]:
# Creamos un arreglos general para todas las imágenes de dimensionamiento (24, 16, 16)
arreglo_imagenes_pernoflojo = []

# Iteramos cada experimento en la lista de experimentos creada previamente
for experimento_i in arreglo_imagenes_por_experimento_pernoflojo:
  # Iteramos en un rango de 64 para usarlo posteriormente
  for idx in range(64):
    # Para cada iteración se crea el arreglo en donde se agregaran las capas correctas de la imagen de dimensionamniento (24, 16, 16)
    imagen = []
    # Iteramos para cada fila en el experimento (son 24 filas)
    for fila in experimento_i:
      # Asociamos por índice los valores de cada lista de imágenes para establecer la relación correcta y agregamos al arreglo creado
      capa_imagen = fila[idx]
      imagen.append(capa_imagen)

    # Transformamos la lista de 24 capas a un arreglo numérico
    arreglo_imagenes_pernoflojo.append(np.array(imagen))

In [40]:
# Revisamos el número de imágenes creadas
len(arreglo_imagenes_pernoflojo)

1280

In [41]:
# Revisamos el dimensionamiento y tipo de un elemento aleatorio en la lista de imágenes
indice_imagen_pernoflojo = randint(0, len(arreglo_imagenes_pernoflojo) - 1)
arreglo_imagenes_pernoflojo[indice_imagen_pernoflojo].shape, type(arreglo_imagenes_pernoflojo[0]), indice_imagen_pernoflojo

((24, 16, 16), numpy.ndarray, 601)

## Datos Réplica

In [42]:
# Se crean 2 listas, una asociada al número de experimento y otra asociada al nivel de white noise presente durante el experimento 
numero_experimentos_replica = []
amplitud_experimentos_replica = []

In [43]:
# Iteramos los nombres de los archivos dentro del conjunto que contiene los datos de fisura
for nombre_archivo in datos_replica:
  # Obtenemos el nombre del experimento
  nombre_experimento = nombre_archivo.split('/')[4]
  # Obtenemos el número del experimento
  numero_experimento = nombre_experimento.split('_')[1]
  # Obtenemos el nivel de white noise del experimento
  amplitud_experimento = nombre_experimento.split('.')[0].split('_')[2].replace('A', '')
  # Agregamos el número del experimento a la lista numero_experimentos_pernoflojo creada anteriormente
  numero_experimentos_replica.append(int(numero_experimento))
  # Agregamos el nivel de white noise del experimento a la lista amplitud_experimentos_pernoflojo creada anteriormente
  if amplitud_experimento == '05':
    amplitud_experimento = 0.5
    amplitud_experimentos_replica.append(amplitud_experimento)
  else:
    amplitud_experimentos_replica.append(int(amplitud_experimento))

In [44]:
# Creamos un DataFrame vacío
df_replica = pd.DataFrame()

In [45]:
# Iteramos por rango la lista datos_fisura
for indice in range(len(datos_replica)):
  # Obtenemos la dirección dentro de la iteración correspondiente
  direccion = datos_replica[indice]
  # Transformamos el .mat a un formato en el que se pueda transformar a DataFrame
  mat = loadmat(direccion)
  df = pd.DataFrame(mat['data'])
  # Para todos los valores del experimento, creamos columnas en donde sus valores se asocian con el nivel de wn y el # de experimento
  df['#_exp'] = numero_experimentos_replica[indice]
  df['amplitud'] = amplitud_experimentos_replica[indice]
  # Colocamos al final del DataFrame df_fisura creado el nuevo DataFrame 
  df_replica = pd.concat([df_replica, df], axis = 0)

In [46]:
# Revisamos el conjunto de datos obtenido para pernoflojo
df_replica.head(8)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,#_exp,amplitud
0,0.000355,0.000146,0.000121,-0.000051,0.000056,0.000194,0.000148,0.000252,0.000287,0.000166,0.000288,0.000208,0.000073,-0.000037,0.000383,0.000238,-0.000084,0.000068,0.000064,0.000306,0.000121,0.000112,-0.000216,-0.000085,1,2.0
1,0.000009,0.000346,0.000240,0.000186,0.000230,0.000027,0.000124,0.000234,0.000363,0.000107,0.000293,0.000239,0.000226,0.000112,0.000393,0.000189,0.000072,0.000068,0.000349,-0.000049,0.000153,0.000133,0.000252,-0.000033,1,2.0
2,0.000136,0.000251,0.000098,0.000175,0.000150,0.000184,0.000024,0.000227,0.000300,0.000195,0.000285,0.000293,0.000143,-0.000009,0.000391,0.000449,0.000009,0.000091,0.000292,0.000251,0.000180,0.000326,0.000021,-0.000019,1,2.0
3,0.000190,0.000126,0.000240,-0.000056,0.000113,0.000203,0.000062,0.000145,0.000325,0.000121,0.000311,0.000234,0.000121,0.000031,0.000378,0.000146,0.000051,0.000073,0.000128,0.000138,0.000127,0.000097,0.000057,-0.000052,1,2.0
4,0.000120,0.000354,0.000238,0.000325,0.000146,0.000057,0.000213,0.000164,0.000299,0.000175,0.000261,0.000269,0.000210,0.000105,0.000377,0.000047,0.000086,0.000059,0.000112,0.000112,0.000163,0.000085,0.000090,-0.000045,1,2.0
5,0.000165,0.000087,0.000249,0.000091,0.000189,0.000228,0.000141,0.000269,0.000243,0.000153,0.000298,0.000194,0.000151,-0.000044,0.000420,0.000306,-0.000079,0.000036,0.000206,0.000246,0.000181,0.000216,-0.000106,-0.000053,1,2.0
6,0.000381,0.000427,0.000296,0.000122,0.000299,0.000188,0.000057,0.000121,0.000329,0.000104,0.000272,0.000298,0.000175,0.000117,0.000428,0.000291,0.000160,0.000079,0.000376,-0.000016,0.000150,0.000189,0.000330,0.000022,1,2.0
7,-0.000010,0.000209,0.000437,0.000280,0.000213,0.000170,0.000087,0.000213,0.000379,0.000220,0.000264,0.000232,0.000239,-0.000024,0.000379,0.000287,-0.000056,0.000125,0.000168,0.000271,0.000246,0.000212,-0.000227,-0.000066,1,2.0


In [47]:
# Si dividimos para 99097 el df_pernoflojo deberíamos obtener 20 experimentos en total
len(df_pernoflojo)/99097

20.0

### Procesamiento de datos

In [48]:
# Establecemos 2 listas, una con los posibles experimentos y otra con los posibles niveles de white noise, lo cual nos ayudará a segmentar los datos
experimentos = [1, 2, 3, 4, 5]
wns = [0.5, 1, 2, 3]
# Creamos una lista en donde pondremos todas las imágenes creadas, segmentadas por experimento y white noise
arreglo_imagenes_por_experimento_replica = []

# Iteramos por experimento y por nivel de white noise
for i in experimentos:
  for j in wns:
    # Creamos filtros que nos permiten identificar el experimento por su número asociado y por su nivel de white noise dentro del conjunto de datos
    filter1 = df_replica["#_exp"] == i
    filter2 = df_replica["amplitud"] == j
    # Filtramos y deshacemos las filas que no corresponden a la búsqueda
    dataset_experimento = df_replica.where(filter1 & filter2).dropna()
    # Creamos arreglo en donde almacenaremos listas de imágenes asociadas a cada columna en un experimento individual
    arreglo_matrices_asociadas = []

    # En el conjunto filtrado, iteramos por cada columna (de la columna 0 a la 23)
    for n in range(24):
      # Establecemos un límite inferior y un límite superior
      indice_inicio = 0
      indice_fin = 256
      # Del conjunto filtrado, obtenemos un subconjunto con muestras cada 6 pasos
      columna_cada_6 = dataset_experimento[n][::6]
      # Creamos una lista en donde guardaremos todas las imágenes creadas dentro de la columna en la que se esta iterando
      matrices_columna = []
    
      # Recorremos en valores de 256 la columna del subconjunto creado para generar las imágenes 16x16 correspondientes 
      while indice_fin < len(columna_cada_6):
        # Guardamos el conjunto de 256 datos dentro de un arreglo
        vector = columna_cada_6[indice_inicio:indice_fin]
        # Redimensionamos el arreglo (de 1x256 a 16x16)
        matriz = vector.to_numpy().reshape((16, 16))
        # Agregamos la imagen dentro de la lista matrices_columna
        matrices_columna.append(matriz)
        # Establecemos el nuevo límite inferior y superior en donde se encontrarán los datos de mi nueva imagen 16x16
        indice_inicio = indice_fin
        indice_fin = indice_fin + 256
      
      # Agregamos las listas de imágenes 16x16 asociadas a una columna a arreglo_matrices_asociadas
      arreglo_matrices_asociadas.append(matrices_columna)
    
    # Agregamos arreglo_matrices_asociadas a arreglos_imagenes_por_experimento, en donde tenemos listas de listas asociadas a un experimento individual
    arreglo_imagenes_por_experimento_replica.append(arreglo_matrices_asociadas)

In [49]:
len(arreglo_imagenes_por_experimento_replica)

20

In [50]:
len(arreglo_imagenes_por_experimento_replica[0])

24

In [51]:
len(arreglo_imagenes_por_experimento_replica[0][0])

64

In [52]:
arreglo_imagenes_por_experimento_replica[0][0][0].size

256

In [53]:
arreglo_imagenes_por_experimento_replica[0][0][0].shape

(16, 16)

In [54]:
# Creamos un arreglos general para todas las imágenes de dimensionamiento (24, 16, 16)
arreglo_imagenes_replica = []

# Iteramos cada experimento en la lista de experimentos creada previamente
for experimento_i in arreglo_imagenes_por_experimento_replica:
  # Iteramos en un rango de 64 para usarlo posteriormente
  for idx in range(64):
    # Para cada iteración se crea el arreglo en donde se agregaran las capas correctas de la imagen de dimensionamniento (24, 16, 16)
    imagen = []
    # Iteramos para cada fila en el experimento (son 24 filas)
    for fila in experimento_i:
      # Asociamos por índice los valores de cada lista de imágenes para establecer la relación correcta y agregamos al arreglo creado
      capa_imagen = fila[idx]
      imagen.append(capa_imagen)

    # Transformamos la lista de 24 capas a un arreglo numérico
    arreglo_imagenes_replica.append(np.array(imagen))

In [55]:
# Revisamos el número de imágenes creadas
len(arreglo_imagenes_replica)

1280

In [56]:
# Revisamos el dimensionamiento y tipo de un elemento aleatorio en la lista de imágenes
indice_imagen_replica = randint(0, len(arreglo_imagenes_replica) - 1)
arreglo_imagenes_replica[indice_imagen_replica].shape, type(arreglo_imagenes_replica[0]), indice_imagen_replica

((24, 16, 16), numpy.ndarray, 137)

## Datos Normal

In [57]:
# Se crean 2 listas, una asociada al número de experimento y otra asociada al nivel de white noise presente durante el experimento 
numero_experimentos_normal = []
amplitud_experimentos_normal = []

In [58]:
# Iteramos los nombres de los archivos dentro del conjunto que contiene los datos de fisura
for nombre_archivo in datos_normal:
  # Obtenemos el nombre del experimento
  nombre_experimento = nombre_archivo.split('/')[4]
  # Obtenemos el número del experimento
  numero_experimento = nombre_experimento.split('_')[1]
  # Obtenemos el nivel de white noise del experimento
  amplitud_experimento = nombre_experimento.split('.')[0].split('_')[2].replace('A', '')
  # Agregamos el número del experimento a la lista numero_experimentos_pernoflojo creada anteriormente
  numero_experimentos_normal.append(int(numero_experimento))
  # Agregamos el nivel de white noise del experimento a la lista amplitud_experimentos_pernoflojo creada anteriormente
  if amplitud_experimento == '05':
    amplitud_experimento = 0.5
    amplitud_experimentos_normal.append(amplitud_experimento)
  else:
    amplitud_experimentos_normal.append(int(amplitud_experimento))

In [59]:
# Creamos un DataFrame vacío
df_normal = pd.DataFrame()

In [60]:
# Iteramos por rango la lista datos_fisura
for indice in range(len(datos_normal)):
  # Obtenemos la dirección dentro de la iteración correspondiente
  direccion = datos_normal[indice]
  # Transformamos el .mat a un formato en el que se pueda transformar a DataFrame
  mat = loadmat(direccion)
  df = pd.DataFrame(mat['data'])
  # Para todos los valores del experimento, creamos columnas en donde sus valores se asocian con el nivel de wn y el # de experimento
  df['#_exp'] = numero_experimentos_normal[indice]
  df['amplitud'] = amplitud_experimentos_normal[indice]
  # Colocamos al final del DataFrame df_fisura creado el nuevo DataFrame 
  df_normal = pd.concat([df_normal, df], axis = 0)

In [61]:
# Revisamos el conjunto de datos obtenido para pernoflojo
df_normal.head(8)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,#_exp,amplitud
0,0.000164,0.000316,0.000257,0.000163,0.000217,0.000159,0.000133,0.000218,0.000316,0.000102,0.000274,0.000244,0.000157,0.000038,0.000348,0.000246,0.000031,0.000106,0.000199,0.000195,0.000147,0.000097,0.000086,-0.000012,1,0.5
1,0.000145,0.000225,0.000230,0.000150,0.000207,0.000144,0.000114,0.000210,0.000329,0.000157,0.000273,0.000263,0.000197,0.000086,0.000392,0.000279,0.000058,0.000102,0.000273,0.000165,0.000207,0.000197,0.000104,0.000011,1,0.5
2,0.000192,0.000263,0.000266,0.000179,0.000199,0.000202,0.000073,0.000173,0.000334,0.000112,0.000264,0.000205,0.000195,0.000054,0.000383,0.000261,0.000009,0.000058,0.000270,0.000182,0.000165,0.000172,0.000084,-0.000006,1,0.5
3,0.000182,0.000258,0.000243,0.000177,0.000252,0.000200,0.000122,0.000200,0.000348,0.000162,0.000270,0.000250,0.000192,0.000021,0.000351,0.000183,0.000024,0.000067,0.000211,0.000210,0.000136,0.000174,0.000068,-0.000037,1,0.5
4,0.000150,0.000261,0.000177,0.000153,0.000195,0.000160,0.000100,0.000196,0.000298,0.000116,0.000289,0.000225,0.000132,0.000028,0.000353,0.000202,-0.000009,0.000075,0.000197,0.000185,0.000117,0.000130,0.000027,-0.000029,1,0.5
5,0.000142,0.000305,0.000240,0.000157,0.000202,0.000155,0.000127,0.000224,0.000341,0.000131,0.000272,0.000254,0.000170,0.000051,0.000403,0.000213,0.000056,0.000096,0.000248,0.000136,0.000183,0.000220,0.000062,-0.000062,1,0.5
6,0.000137,0.000227,0.000229,0.000136,0.000189,0.000139,0.000080,0.000173,0.000335,0.000158,0.000237,0.000250,0.000213,0.000045,0.000417,0.000219,-0.000008,0.000092,0.000242,0.000146,0.000136,0.000185,0.000022,0.000015,1,0.5
7,0.000138,0.000244,0.000208,0.000149,0.000181,0.000118,0.000099,0.000144,0.000282,0.000162,0.000263,0.000242,0.000211,0.000035,0.000382,0.000178,0.000087,0.000085,0.000183,0.000130,0.000100,0.000147,0.000056,-0.000007,1,0.5


In [62]:
# Si dividimos para 99097 el df_pernoflojo deberíamos obtener 20 experimentos en total NO SABEMOS TODAVIA
len(df_normal)/99097

40.0

### Procesamiento de datos

In [74]:
# Establecemos 2 listas, una con los posibles experimentos y otra con los posibles niveles de white noise, lo cual nos ayudará a segmentar los datos
experimentosn = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
wns = [0.5, 1, 2, 3]
# Creamos una lista en donde pondremos todas las imágenes creadas, segmentadas por experimento y white noise
arreglo_imagenes_por_experimento_normal = []

# Iteramos por experimento y por nivel de white noise
for i in experimentosn:
  for j in wns:
    # Creamos filtros que nos permiten identificar el experimento por su número asociado y por su nivel de white noise dentro del conjunto de datos
    filter1 = df_normal["#_exp"] == i
    filter2 = df_normal["amplitud"] == j
    # Filtramos y deshacemos las filas que no corresponden a la búsqueda
    dataset_experimento = df_normal.where(filter1 & filter2).dropna()
    # Creamos arreglo en donde almacenaremos listas de imágenes asociadas a cada columna en un experimento individual
    arreglo_matrices_asociadas = []

    # En el conjunto filtrado, iteramos por cada columna (de la columna 0 a la 23)
    for n in range(24):
      # Establecemos un límite inferior y un límite superior
      indice_inicio = 0
      indice_fin = 256
      # Del conjunto filtrado, obtenemos un subconjunto con muestras cada 6 pasos
      columna_cada_6 = dataset_experimento[n][::6]
      # Creamos una lista en donde guardaremos todas las imágenes creadas dentro de la columna en la que se esta iterando
      matrices_columna = []
    
      # Recorremos en valores de 256 la columna del subconjunto creado para generar las imágenes 16x16 correspondientes 
      while indice_fin < len(columna_cada_6):
        # Guardamos el conjunto de 256 datos dentro de un arreglo
        vector = columna_cada_6[indice_inicio:indice_fin]
        # Redimensionamos el arreglo (de 1x256 a 16x16)
        matriz = vector.to_numpy().reshape((16, 16))
        # Agregamos la imagen dentro de la lista matrices_columna
        matrices_columna.append(matriz)
        # Establecemos el nuevo límite inferior y superior en donde se encontrarán los datos de mi nueva imagen 16x16
        indice_inicio = indice_fin
        indice_fin = indice_fin + 256
      
      # Agregamos las listas de imágenes 16x16 asociadas a una columna a arreglo_matrices_asociadas
      arreglo_matrices_asociadas.append(matrices_columna)
    
    # Agregamos arreglo_matrices_asociadas a arreglos_imagenes_por_experimento, en donde tenemos listas de listas asociadas a un experimento individual
    arreglo_imagenes_por_experimento_normal.append(arreglo_matrices_asociadas)

In [75]:
len(arreglo_imagenes_por_experimento_normal)

40

In [76]:
len(arreglo_imagenes_por_experimento_normal[0])

24

In [77]:
len(arreglo_imagenes_por_experimento_normal[0][0])

64

In [78]:
arreglo_imagenes_por_experimento_normal[0][0][0].size

256

In [79]:
arreglo_imagenes_por_experimento_normal[0][0][0].shape

(16, 16)

In [80]:
# Creamos un arreglos general para todas las imágenes de dimensionamiento (24, 16, 16)
arreglo_imagenes_normal = []

# Iteramos cada experimento en la lista de experimentos creada previamente
for experimento_i in arreglo_imagenes_por_experimento_normal:
  # Iteramos en un rango de 64 para usarlo posteriormente
  for idx in range(64):
    # Para cada iteración se crea el arreglo en donde se agregaran las capas correctas de la imagen de dimensionamniento (24, 16, 16)
    imagen = []
    # Iteramos para cada fila en el experimento (son 24 filas)
    for fila in experimento_i:
      # Asociamos por índice los valores de cada lista de imágenes para establecer la relación correcta y agregamos al arreglo creado
      capa_imagen = fila[idx]
      imagen.append(capa_imagen)

    # Transformamos la lista de 24 capas a un arreglo numérico
    arreglo_imagenes_normal.append(np.array(imagen))

In [81]:
# Revisamos el número de imágenes creadas
len(arreglo_imagenes_normal)

2560

In [82]:
# Revisamos el dimensionamiento y tipo de un elemento aleatorio en la lista de imágenes
indice_imagen_normal = randint(0, len(arreglo_imagenes_normal) - 1)
arreglo_imagenes_normal[indice_imagen_normal].shape, type(arreglo_imagenes_normal[0]), indice_imagen_normal

((24, 16, 16), numpy.ndarray, 95)

In [85]:
# Creamos los conjuntos generales de imágenes asociados a fallo (fisura y pernoflojo) y sano (normal y réplica)
arreglo_imagenes_fallos = arreglo_imagenes_fisura + arreglo_imagenes_pernoflojo
arreglo_imagenes_sano = arreglo_imagenes_normal + arreglo_imagenes_replica

In [86]:
len(arreglo_imagenes_fallos), len(arreglo_imagenes_sano)

(2560, 3840)

In [87]:
len(arreglo_imagenes_sano) - len(arreglo_imagenes_fallos)

1280

In [92]:
arreglo_imagenes_fallos_auxiliar = []

for i in range(len(arreglo_imagenes_sano) - len(arreglo_imagenes_fallos)):
  arreglo_imagenes_fallos_auxiliar.append(random.choice(arreglo_imagenes_fallos))

In [93]:
len(arreglo_imagenes_fallos_auxiliar)

1280

## Red Neuronal Siamesa
Tenemos:
- arreglo_imagenes_fallos
- arreglo_imagenes_sano

In [95]:
def crear_pares(arreglo_imagenes_sano, arreglo_imagenes_fallos, arreglo_imagenes_fallos_auxiliar):
  """
  Función empleada para crear los pares de imágenes que se utilizan
  para el entrenamiento de la SNN.
  """
  print("Creando pares...")
  # Creamos arreglo en donde se insertarán los pares (positivos y negativos)
  pares_img = []
  # Creamos arreglo en donde se insertarán la clasificación de los pares (similares = 1, no similares = 0)
  pares_label = []

  # Recorremos el arreglo de imágenes de fisura y por cada imagen recorrida creamos un par positivo y uno negativo
  for imagen_sano in arreglo_imagenes_sano:

    # La imagen positiva será un dato aleatorio perteneciente al arreglo de la misma clase (fisura)
    imagen_positiva_sano = random.choice(arreglo_imagenes_sano)
    # La imagen negativa será un dato aleatorio perteneciente al arreglo de la otra clase (perno flojo)
    imagen_negativa_sano = random.choice(arreglo_imagenes_fallos)

    # Agregamos el par de imagenes positivas al arreglo de pares de imagenes
    pares_img.append([imagen_sano, imagen_positiva_sano])
    # Agregamos el label correspondiente a un par de imágenes positivas (1)
    pares_label.append([1])

    # Agregamos el par de imagenes negativas al arreglo de pares de imagenes
    pares_img.append([imagen_sano, imagen_negativa_sano])
    # Agregamos el label correspondiente a un par de imágenes negativas (0)
    pares_label.append([0])

  # Recorremos el arreglo de imágenes de fisura y por cada imagen recorrida creamos un par positivo y uno negativo
  for imagen_fallos in arreglo_imagenes_fallos:

    # La imagen positiva será un dato aleatorio perteneciente al arreglo de la misma clase (perno flojo)
    imagen_positiva_fallos = random.choice(arreglo_imagenes_fallos)
    # La imagen negativa será un dato aleatorio perteneciente al arreglo de la otra clase (fisura)
    imagen_negativa_fallos = random.choice(arreglo_imagenes_sano)

    # Agregamos el par de imagenes positivas al arreglo de pares de imagenes
    pares_img.append([imagen_fallos, imagen_positiva_fallos])
    # Agregamos el label correspondiente a un par de imágenes positivas (1)
    pares_label.append([1])

    # Agregamos el par de imagenes negativas al arreglo de pares de imagenes
    pares_img.append([imagen_fallos, imagen_negativa_fallos])
    # Agregamos el label correspondiente a un par de imágenes negativas (0)
    pares_label.append([0])
  
  for imagen_fallos_auxiliar in arreglo_imagenes_fallos_auxiliar:

    # La imagen positiva será un dato aleatorio perteneciente al arreglo de la misma clase (perno flojo)
    imagen_positiva_fallos_auxiliar = random.choice(arreglo_imagenes_fallos)
    # La imagen negativa será un dato aleatorio perteneciente al arreglo de la otra clase (fisura)
    imagen_negativa_fallos_auxiliar = random.choice(arreglo_imagenes_sano)

    # Agregamos el par de imagenes positivas al arreglo de pares de imagenes
    pares_img.append([imagen_fallos_auxiliar, imagen_positiva_fallos_auxiliar])
    # Agregamos el label correspondiente a un par de imágenes positivas (1)
    pares_label.append([1])

    # Agregamos el par de imagenes negativas al arreglo de pares de imagenes
    pares_img.append([imagen_fallos_auxiliar, imagen_negativa_fallos_auxiliar])
    # Agregamos el label correspondiente a un par de imágenes negativas (0)
    pares_label.append([0])

  print("Pares creados.")

  return (np.array(pares_img), np.array(pares_label))

In [96]:
(pares, labels) = crear_pares(arreglo_imagenes_sano, arreglo_imagenes_fallos, arreglo_imagenes_fallos_auxiliar)

Creando pares...
Pares creados.


In [97]:
len(pares), len(labels)

(15360, 15360)

In [ ]:
# 3840 para sano total
# 2560 para fallo + 1280 de fallo auxiliar = 3840 para fallo total
# 3840 sano total + 3840 fallo total = 7680 imagenes totales
# 2 pares por imagens (un par positivo + un par negativo = 2 pares)
# 7680 imagenes totales x 2 pares por cada imagen = 15360 pares de imágenes

### Segmentación en entrenamiento, validación y prueba

In [ ]:
np.random.seed(300)

pares_aleatorizados = pares
np.random.shuffle(pares_aleatorizados)

In [ ]:
np.random.seed(300)

labels_aleatorizados = labels
np.random.shuffle(labels_aleatorizados)

In [ ]:
# Obtenemos el 80%, 70%, 15% y 10% de los valores totales
porcentaje_80 = int(len(pares_aleatorizados)*0.8)
porcentaje_70 = int(len(pares_aleatorizados)*0.7)
porcentaje_15 = int(len(pares_aleatorizados)*0.15)
porcentaje_10 = int(len(pares_aleatorizados)*0.10)

porcentaje_80, porcentaje_70, porcentaje_15, porcentaje_10

### Creación de SNN

In [ ]:
def euclidean_distance(vectors):
  
  # Descomprimimos los vectores en listas separadas
  (featsA, featsB) = vectors

  # Realizamos la suma de las diferenias cuadradas entre vectores
  sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)

  # Retornamos la distancia euclidiana
  return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [ ]:
def graficar_curva_perdidas(history):
  """
  Función utilizada para visualizar la historia de nuestro modelo.
  Incluye pérdidas y precisión a lo largo de cada iteración.
  """
  # Obtenemos las pérdidas de los conjuntos de entrenamiento y validación
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  # Obtenemos la precisión de los conjuntos de entrenamiento y validación
  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  # Determinamos las iteraciones que hubo durante el entrenamiento
  epochs = range(len(history.history['loss']))

  # Graficamos las pérdidas de ambos conjuntos a lo largo de las iteraciones
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('loss')
  plt.xlabel('epochs')
  plt.legend()

  # Graficamos la precisión de ambos conjuntos a lo largo de las iteraciones
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('accuracy')
  plt.xlabel('epochs')
  plt.legend()

Modelo Experimental #1

- entrenamiento = 80%
- validación = 10%
- prueba = 10%

In [ ]:
# Definimos el conjunto de entrenamiento
me1_pares_entrenamiento = pares_aleatorizados[:porcentaje_80]
me1_labels_entrenamiento = labels_aleatorizados[:porcentaje_80]

# Definimos el conjunto de validación
me1_pares_validacion = pares_aleatorizados[porcentaje_80:porcentaje_80+porcentaje_10]
me1_labels_validacion = labels_aleatorizados[porcentaje_80:porcentaje_80+porcentaje_10]

# Definimos el conjunto de prueba
me1_pares_prueba = pares_aleatorizados[porcentaje_80+porcentaje_10:]
me1_labels_prueba = labels_aleatorizados[porcentaje_80+porcentaje_10:]

In [ ]:

# Creamos función que retorna la estructura de la red convolucional que se va a utilizar
def build_siamese_model_1(input_shape):
  model = tf.keras.Sequential([
                                tf.keras.layers.Conv2D(filters=64,
                                                        kernel_size=10,
                                                        activation='relu',
                                                        padding='same',
                                                        input_shape=input_shape),
                                tf.keras.layers.MaxPooling2D(pool_size=2),
                                tf.keras.layers.Conv2D(filters=128,
                                                        kernel_size=7,
                                                        activation='relu',
                                                        padding='same'),
                                tf.keras.layers.MaxPooling2D(pool_size=2),
                                tf.keras.layers.Conv2D(filters=128,
                                                        kernel_size=4,
                                                        activation='relu',
                                                        padding='same'),
                                tf.keras.layers.MaxPooling2D(pool_size=2),
                                tf.keras.layers.Conv2D(filters=256,
                                                        kernel_size=4,
                                                        activation='relu',
                                                        padding='same'),
                                tf.keras.layers.Flatten(),
                                tf.keras.layers.Dense(4960)
  ])

  return model

In [ ]:
tf.random.set_seed(100)

# Creamos el modelo general
IMG_SHAPE = (24, 16, 16)

# Definimos los inputs (imágenes) que ingresan a cada CNN
imgA = tf.keras.Input(IMG_SHAPE)
imgB = tf.keras.Input(IMG_SHAPE)

# Creamos el featureExtractor, el cual obtiene las características principales de las imágenes ingresadas y las muestra en un vector
featureExtractor = build_siamese_model_1(input_shape=IMG_SHAPE)

# Ingresamos cada imagen al featureExtractor
featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)

# Creamos una capa Lambda (que aplica una función sobre los datos) para obtener la distancia euclidiana
distancia = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])

# Creamos la última capa asociada a la función de activación sigmoide
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(distancia)

# Establecemos el modelo general
model_1 = tf.keras.Model(inputs=[imgA, imgB], outputs=outputs)

# Empleamos el loss como binary_crossentropy porque solo queremos saber si las imágenes son similares o no (no queremos clasificarlas)
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                metrics=['accuracy'])

# Hacemos el fit del modelo
history_1 = model_1.fit(
    [me1_pares_entrenamiento[:, 0], me1_pares_entrenamiento[:, 1]], me1_labels_entrenamiento[:],
     validation_data=([me1_pares_validacion[:, 0], me1_pares_validacion[:, 1]], me1_labels_validacion[:]),
     batch_size=32,
     epochs=5
)

In [ ]:
graficar_curva_perdidas(history_1)


Modelo Experimental #2

- entrenamiento = 80%
- validación = 15%
- prueba = 5%

In [ ]:
# Definimos el conjunto de entrenamiento
me2_pares_entrenamiento = pares_aleatorizados[:porcentaje_80]
me2_labels_entrenamiento = labels_aleatorizados[:porcentaje_80]

# Definimos el conjunto de validación
me2_pares_validacion = pares_aleatorizados[porcentaje_80:porcentaje_80+porcentaje_15]
me2_labels_validacion = labels_aleatorizados[porcentaje_80:porcentaje_80+porcentaje_15]

# Definimos el conjunto de prueba
me2_pares_prueba = pares_aleatorizados[porcentaje_80+porcentaje_15:]
me2_labels_prueba = labels_aleatorizados[porcentaje_80+porcentaje_15:]

In [ ]:
def build_siamese_model_2(input_shape):
  model = tf.keras.Sequential([
                                tf.keras.layers.Conv2D(filters=64,
                                                        kernel_size=10,
                                                        activation='relu',
                                                        padding='same',
                                                        input_shape=input_shape),
                                tf.keras.layers.MaxPooling2D(pool_size=2),
                                tf.keras.layers.Conv2D(filters=128,
                                                        kernel_size=7,
                                                        activation='relu',
                                                        padding='same'),
                                tf.keras.layers.MaxPooling2D(pool_size=2),
                                tf.keras.layers.Conv2D(filters=128,
                                                        kernel_size=4,
                                                        activation='relu',
                                                        padding='same'),
                                tf.keras.layers.MaxPooling2D(pool_size=2),
                                tf.keras.layers.Conv2D(filters=256,
                                                        kernel_size=4,
                                                        activation='relu',
                                                        padding='same'),
                                tf.keras.layers.Flatten(),
                                tf.keras.layers.Dense(4960)
  ])

  return model

In [ ]:
tf.random.set_seed(100)

IMG_SHAPE = (24, 16, 16)

imgA = tf.keras.Input(IMG_SHAPE)
imgB = tf.keras.Input(IMG_SHAPE)

featureExtractor = build_siamese_model_2(input_shape=IMG_SHAPE)

featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)

distancia = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(distancia)

model_2 = tf.keras.Model(inputs=[imgA, imgB], outputs=outputs)

model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                metrics=['accuracy'])

history_2 = model_2.fit(
    [me2_pares_entrenamiento[:, 0], me2_pares_entrenamiento[:, 1]], me2_labels_entrenamiento[:],
     validation_data=([me2_pares_validacion[:, 0], me2_pares_validacion[:, 1]], me2_labels_validacion[:]),
     batch_size=32,
     epochs=5
)

In [ ]:
graficar_curva_perdidas(history_2)

Modelo Experimental #3

- entrenamiento = 70%
- validación = 15%
- prueba = 15%

In [ ]:
# Definimos el conjunto de entrenamiento
me3_pares_entrenamiento = pares_aleatorizados[:porcentaje_70]
me3_labels_entrenamiento = labels_aleatorizados[:porcentaje_70]

# Definimos el conjunto de validación
me3_pares_validacion = pares_aleatorizados[porcentaje_70:porcentaje_70+porcentaje_15]
me3_labels_validacion = labels_aleatorizados[porcentaje_70:porcentaje_70+porcentaje_15]

# Definimos el conjunto de prueba
me3_pares_prueba = pares_aleatorizados[porcentaje_70+porcentaje_15:]
me3_labels_prueba = labels_aleatorizados[porcentaje_70+porcentaje_15:]

In [ ]:
def build_siamese_model_3(input_shape):
  model = tf.keras.Sequential([
                                tf.keras.layers.Conv2D(filters=64,
                                                        kernel_size=10,
                                                        activation='relu',
                                                        padding='same',
                                                        input_shape=input_shape),
                                tf.keras.layers.MaxPooling2D(pool_size=2),
                                tf.keras.layers.Conv2D(filters=128,
                                                        kernel_size=7,
                                                        activation='relu',
                                                        padding='same'),
                                tf.keras.layers.MaxPooling2D(pool_size=2),
                                tf.keras.layers.Conv2D(filters=128,
                                                        kernel_size=4,
                                                        activation='relu',
                                                        padding='same'),
                                tf.keras.layers.MaxPooling2D(pool_size=2),
                                tf.keras.layers.Conv2D(filters=256,
                                                        kernel_size=4,
                                                        activation='relu',
                                                        padding='same'),
                                tf.keras.layers.Flatten(),
                                tf.keras.layers.Dense(4960)
  ])

  return model

In [ ]:
tf.random.set_seed(100)

IMG_SHAPE = (24, 16, 16)

imgA = tf.keras.Input(IMG_SHAPE)
imgB = tf.keras.Input(IMG_SHAPE)

featureExtractor = build_siamese_model_3(input_shape=IMG_SHAPE)

featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)

distancia = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(distancia)

model_3 = tf.keras.Model(inputs=[imgA, imgB], outputs=outputs)

model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                metrics=['accuracy'])

history_3 = model_3.fit(
    [me3_pares_entrenamiento[:, 0], me3_pares_entrenamiento[:, 1]], me3_labels_entrenamiento[:],
     validation_data=([me3_pares_validacion[:, 0], me3_pares_validacion[:, 1]], me3_labels_validacion[:]),
     batch_size=32,
     epochs=30
)

In [ ]:
graficar_curva_perdidas(history_3)

### Matrices de confusion

In [ ]:
# Creamos una función que nos permita visualizar la matriz de confusión de los resultados
sns.set(font_scale=1.5)

def plot_conf_mat(y_test, y_preds):
    
    fig, ax = plt.subplots(figsize=(4,4))
    ax = sns.heatmap(confusion_matrix(y_test, y_preds),
                     annot=True,
                     cbar=False,
                     fmt='g')
    plt.xlabel('Label Reales')
    plt.ylabel('Predicciones')

In [ ]:
# Guardamos las predicciones en una variable
predicciones_1 = model_1.predict([me1_pares_prueba[:, 0], me1_pares_prueba[:, 1]])

In [ ]:
# Redondeamos los valores predecidos para poder compararlos con los valores reales
for i in range(len(predicciones_1)):
  predicciones_1[i][0] = round(predicciones_1[i][0])

In [ ]:
# Graficamos la matriz de confusión
plot_conf_mat(me1_labels_prueba, predicciones_1)

In [ ]:
# Guardamos las predicciones en una variable
predicciones_2 = model_2.predict([me2_pares_prueba[:, 0], me2_pares_prueba[:, 1]])

In [ ]:
# Redondeamos los valores predecidos para poder compararlos con los valores reales
for i in range(len(predicciones_2)):
  predicciones_2[i][0] = round(predicciones_2[i][0]

In [ ]:
# Graficamos la matriz de confusión
plot_conf_mat(me2_labels_prueba, predicciones_2)

In [ ]:
# Guardamos las predicciones en una variable
predicciones_3 = model_3.predict([me3_pares_prueba[:, 0], me3_pares_prueba[:, 1]])

In [ ]:
# Redondeamos los valores predecidos para poder compararlos con los valores reales
for i in range(len(predicciones_3)):
  predicciones_3[i][0] = round(predicciones_3[i][0])

In [ ]:
# Graficamos la matriz de confusión
plot_conf_mat(me3_labels_prueba, predicciones_3)

### Guardado del modelo

In [ ]:
# Creamos la función de guardado del modelo
def save_model(model, suffix=None):
  """
  Guarda un modelo en el directorio de modelos y agrega un string.
  """
  # Creamos el pathname del directorio del modelo con el tiempo actual
  model_dir = os.path.join('drive/MyDrive/modelos',
                           datetime.datetime.now().strftime('%Y%m%d-%H%M%s'))
  # Guardado con formato de modelo
  model_path = model_dir + '-' + suffix + '.h5'
  print(f'Guardando modelo en: {model_path}...')
  model.save(model_path)
  return model_path

In [ ]:
modelo_1_path = save_model(model=model_1, suffix='modelo_SNN_TurbinaJacket_fisura_pernoflojo_2')